In [1]:
import urllib.request
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd

link = 'https://rally-base.com/2023/croatia-rally-2023/?ssId='
startat, no_ss=7447, int(20) # starting number of url, count of Special Stages
canceled = []

stages = [number for number in range(no_ss)]


rally_23 = []

In [2]:
if canceled:
    for j in canceled: stages.remove(j-1)
#print(stages)


In [3]:
for ss in stages:
    val= startat + ss
    ss_a = str(val)
    my_url11 = link + ss_a
    #print(ss, val, ss_a, "\n", my_url11)
    
    req = urllib.request.Request(my_url11, headers={'User-Agent': 'Mozilla/5.0'})
    uClient11 = urllib.request.urlopen(req)
    page_html11 = uClient11.read()
    uClient11.close()
    data = pd.read_html(page_html11)[1]
   
    data.columns = data.iloc[0]
    data = data[1:]
    data['ss']=ss+1
    #print(data.columns)
    
    equal = '-' in data['Pos.'].unique()
    if equal:
        data['Pos.'] = data['Pos.'].replace('-', method='ffill')
    
    data.to_csv('01_rally23_SS'+str(ss+1)+'.csv', index=False)

    rally_23.append(data)

In [4]:
rally2023_stages = pd.concat(rally_23, axis=0)
#rally2023_stages['Pos.'] = rally2023_stages['Pos.'].astype(str).astype(int)
rally2023_stages['No.'] = rally2023_stages['No.'].astype(str).astype(int)
#rally2023_stages.to_csv('03_rally2023.csv', index=False)
rally2023_stages = rally2023_stages.fillna("-")
rally2023_temp=rally2023_stages.drop(['Group','SS time', 'Aver. speed sec/km', 'Diff.Leader Diff.Prev.'], axis=1)
rally2023_view = rally2023_temp.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')

rally2023_temp2 = rally2023_temp[(rally2023_temp['No.']<30)]
rally2023_temp3 = rally2023_temp[(rally2023_temp['No.']==69)]
#rally2023_temp2.append(rally2023_temp3)
#rally2023_temp4 = rally2023_temp2.append(rally2023_temp3)
rally2023_temp4 = pd.concat([rally2023_temp2, rally2023_temp3],axis=0)
rally2023_view2 = rally2023_temp4.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')
rally2023_view2 = rally2023_view2.fillna("-")
rally2023_view2

0                                                      Pos.                  \
ss                                                       1   2   3   4   5    
No. Driver / Co-driver Vehicle                                                
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...    6   3   6   5   6   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...    8   6   7   8   7   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...    3   4   4   2   3   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...    2   1   2   4   5   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...    1  17   1   1   1   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...    7   5   5   7   8   
20  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...   12  11  16  12  13   
21  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...    9   7  12  16  10   
22  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...   10  24  10   9  12   
23  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...   11  41  11  11   9   
24  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...   13  12   9  10  11   
25  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...   14  27  15  14  14   
26  Pajari  Sami  /  Mälkönen  Enni  Škoda Fabia RS...   16  10  14  15  17   
27  Cais  Erik  /  Těšínský  P.  Škoda Fabia RS Rally2   52  51  51  51  49   
28  Munster  G.  /  Louka  Louis  Ford Fiesta Rally...   20  14  17  29  18   
29  Linnamäe  Georg  /  Morgan  James  Hyundai i20 ...   17  13  19  17  49   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...    4  29   8   6   3   

0                                                                           \
ss                                                      6   7   8   9   10   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   1   7   6   7   8   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...   9   8   8   6   7   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   6   1   3   2   3   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...   4   2   5   4   4   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...   3   4   7   3   2   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...  11   6   4   8   6   
20  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...  14  14  12  16  16   
21  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...   7  15  16  11  13   
22  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...  13  12  10  10  14   
23  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...  16  10  12  14  10   
24  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...  10  11   9  13   9   
25  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...  15  16  17   9  11   
26  Pajari  Sami  /  Mälkönen  Enni  Škoda Fabia RS...  17  13  14  15  15   
27  Cais  Erik  /  Těšínský  P.  Škoda Fabia RS Rally2  47  47  47  17  17   
28  Munster  G.  /  Louka  Louis  Ford Fiesta Rally...  47  47  47  20  20   
29  Linnamäe  Georg  /  Morgan  James  Hyundai i20 ...  47  47  47  18  18   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...   5   4   2   1   1   

0                                                                           \
ss                                                      11  12  13  14  15   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   6   4   7   5   4   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...   7   7   4   7   7   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   4   5   2   1   5   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...  50  50  48  47  47   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...   2   1   1   3   2   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...   5   6   6   6   6   
20  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...  21  12  14  14   9   
21  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...  13   9  13  11  13   


In [5]:
pd.set_option("display.max_columns", None)
rally2023_view2

0                                                      Pos.                  \
ss                                                       1   2   3   4   5    
No. Driver / Co-driver Vehicle                                                
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...    6   3   6   5   6   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...    8   6   7   8   7   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...    3   4   4   2   3   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...    2   1   2   4   5   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...    1  17   1   1   1   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...    7   5   5   7   8   
20  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...   12  11  16  12  13   
21  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...    9   7  12  16  10   
22  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...   10  24  10   9  12   
23  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...   11  41  11  11   9   
24  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...   13  12   9  10  11   
25  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...   14  27  15  14  14   
26  Pajari  Sami  /  Mälkönen  Enni  Škoda Fabia RS...   16  10  14  15  17   
27  Cais  Erik  /  Těšínský  P.  Škoda Fabia RS Rally2   52  51  51  51  49   
28  Munster  G.  /  Louka  Louis  Ford Fiesta Rally...   20  14  17  29  18   
29  Linnamäe  Georg  /  Morgan  James  Hyundai i20 ...   17  13  19  17  49   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...    4  29   8   6   3   

0                                                                           \
ss                                                      6   7   8   9   10   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   1   7   6   7   8   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...   9   8   8   6   7   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   6   1   3   2   3   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...   4   2   5   4   4   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...   3   4   7   3   2   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...  11   6   4   8   6   
20  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...  14  14  12  16  16   
21  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...   7  15  16  11  13   
22  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...  13  12  10  10  14   
23  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...  16  10  12  14  10   
24  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...  10  11   9  13   9   
25  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...  15  16  17   9  11   
26  Pajari  Sami  /  Mälkönen  Enni  Škoda Fabia RS...  17  13  14  15  15   
27  Cais  Erik  /  Těšínský  P.  Škoda Fabia RS Rally2  47  47  47  17  17   
28  Munster  G.  /  Louka  Louis  Ford Fiesta Rally...  47  47  47  20  20   
29  Linnamäe  Georg  /  Morgan  James  Hyundai i20 ...  47  47  47  18  18   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...   5   4   2   1   1   

0                                                                           \
ss                                                      11  12  13  14  15   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   6   4   7   5   4   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...   7   7   4   7   7   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   4   5   2   1   5   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...  50  50  48  47  47   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...   2   1   1   3   2   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...   5   6   6   6   6   
20  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...  21  12  14  14   9   
21  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...  13   9  13  11  13   
